Выбран сайт https://www.e-katalog.ru/
- Все ссылки с сайта скачены в файл pos_rew.txt
- Далее парсим с сайта отзывы
- положительные отзывы в папку pos
- негативные в папку neg
- так же сам отзыв по папкам от 1 до 4 в зависимости от оценки поставленой пользователем(не использовались)

In [5]:
import requests
import bs4
import os
import time
import numpy as np
import pickle
from itertools import product
import matplotlib.pyplot as plt
%matplotlib inline

In [26]:
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer,TfidfTransformer
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
import glob
import pandas as pd
from sklearn.metrics import accuracy_score

In [30]:
# review parsing price&reviews
# Счетчик отзывов
counter = 0
# Загружаем отобранные ссылки из файла
with open("./products/pos_rew.txt") as links_file:
    for link in links_file:
        # На всякий случай делаем случайную задержку перед каждым парсингом
        t_sleep = np.random.randint(1, 100)*0.01
        time.sleep(t_sleep)
        # читаем данные по ссылке
        req = requests.get(link)
        link_parser = bs4.BeautifulSoup(req.text, 'lxml')
        # Находим все отзывы на странцие
        reviews_parser = link_parser.find('div',attrs = {'class', 'common-table-div s-width'})
#         print reviews_parser
        # Парсим следующий отзыв
        if reviews_parser != None:
             review_parser = reviews_parser.findAll('td', attrs = {'class', 'review-td'})          
        if review_parser != None:
#             print review_parser
            for res in review_parser:
                counter += 1
                
                pic = res.findChild('div', attrs = {'class', 'review-plus'})
                # Рассматриваем отмеченные достоинства как отдельный положительный отзыв
                # Записываем его в папку ./pos
                if pic != None:
                    with open("./products/reviews/pos/"+str(counter)+".txt", 'w') as fid:
                        fid.write(pic.text.strip().lower().encode('utf8'))
                
                pic1 = res.findChild('div', attrs = {'class', 'review-minus'})
                # Рассматриваем отмеченные недостатки как отдельный негативный отзыв
                # Записываем его в папку ./neg
                if pic1 != None:
                    with open("./products/reviews/neg/"+str(counter)+".txt", 'w') as fid:
                        fid.write(pic1.text.strip().lower().encode('utf8'))
 

                    
                rat = res.findAll('img')
#                 print rat
                for i in rat:
                    if str(i) == '<img alt="" src="/img/svg/review-smile-1.svg"/>':
                        rating = 1

                    elif str(i) == '<img alt="" src="/img/svg/review-smile-2.svg"/>':
                        rating = 2

                    elif str(i) == '<img alt="" src="/img/svg/review-smile-3.svg"/>':
                        rating = 3

                    elif str(i) == '<img alt="" src="/img/svg/review-smile-4.svg"/>':
                        rating = 4
                        
                pic2 = res.findChild('span', attrs = {'property':"v:description"})
                if pic2 != None:
                    with open("./products/reviews/rating/"+str(rating)+"/"+str(counter)+".txt", 'w') as fid:
                        fid.write(pic2.text.strip().lower().encode('utf8'))

Парсим тест

In [7]:
with open("test.csv", "r") as fid:
    text = fid.read()

In [8]:
parse_test = bs4.BeautifulSoup(text, 'lxml')
cnt = 0
data_test = []
for review in parse_test.findAll('review'):
    cnt += 1
    print cnt
    print review.text
    data_test.append(review.text)    

1
Ужасно слабый аккумулятор, это основной минус этого аппарата, разряжается буквально за пару часов при включенном wifi и на макс подсветке, например если играть или смотреть видео, следовательно использовать можно только если есть постоянная возможность подзарядиться. Качества звука через динамик далеко не на высоте.Наблюдаются незначительные тормоза в некоторых приложениях и вообще в меню. Очень мало встроенной памяти, а приложения устанавливаются именно туда, с этим связанны неудобства - нужно постоянно переносить их на карту памяти.
Несколько неудобно что нету отдельной кнопки для фото. Подумываю купить батарею большей емкость мб что нибудь измениться.

2
т.е.  вы можете видеть всю ветку беседы с конкретным человеком , а не искать в куче входящих смс с нужной информацией.
работает чисто без галюнов,падал,шврялся,купался,используется как тел и плеер,в метро поездов не слышно уши юзаю сенхайзер 300 и 500,mp3 звонок чистый без сипов.Экран  не горилла глас , но на солнце сохраняет чита

## Парсим негативные коментарии и размечаем

In [9]:
allFiles = glob.glob('./products/reviews/neg/*.txt')
data = []
for i in allFiles:
    with open(str(i), "rb") as f:
        contents = f.read().decode("UTF-8")
        data.append(contents)

In [10]:
df_1n = pd.DataFrame(data, columns = ['data'])

In [11]:
target = [int(0) for i in range(0,len(df_1n))]
t_1n = pd.DataFrame(target,columns = ['target'])
    
    

In [12]:
neg = pd.concat([t_1n, df_1n], axis=1)
len(neg)

2499

## Парсим позитивные коментарии и размечаем

In [13]:
allFiles = glob.glob('./products/reviews/pos/*.txt')
data = []
for i in allFiles:                     
    with open(str(i), "rb") as f:
        contents = f.read().decode("UTF-8")
        data.append(contents)

In [14]:
df_2p = pd.DataFrame(data, columns = ['data'])
target = [int(1) for i in range(0,len(df_2p))]
t_2p = pd.DataFrame(target,columns = ['target'])

In [15]:
pos = pd.concat([t_2p, df_2p], axis=1)
len(pos)

2499

## объединяем данные

In [16]:
ddata = pd.concat([pos,neg],ignore_index=True)
len(ddata)

4998

In [17]:
df = ddata.sample(frac=1).reset_index(drop=True)

In [18]:
data = df
print "\nРазмер датасета: ", len(data.target)

# 0 - neg, 1 - pos
print "\nПример данных"
print "Label:", data.target[1]

print "Text:", data.data[1]

print "\nДоля положительных отзывов: ", data.target.mean()


Размер датасета:  4998

Пример данных
Label: 0
Text: когда только купил выдовало ошибку, позже, через недели 2 начал отклеиваться тач скрин, за время эксплотации очень часто нагревался, играя игры, смотря видео может спокойно залагать и все, вам надо будет подождать минут 5 чтобы отлагал.

Доля положительных отзывов:  0.5


Делаем пайплан, все эксперементы я убрал, что бы код был короче

In [19]:
pipe = Pipeline([('Vectorizer', TfidfVectorizer(ngram_range=(3,7), analyzer='char_wb') ), ('Classifier',  LogisticRegression())])
scores = cross_val_score(pipe, data.data, data.target, cv=5)
scores.mean()

0.8863523046092185

In [22]:
pipe.fit(data.data, data.target)

Pipeline(memory=None,
     steps=[('Vectorizer', TfidfVectorizer(analyzer='char_wb', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(3, 7), norm=u'l2', preprocessor=None, smooth_...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [23]:
with open("./products/labels.txt", 'r') as fid:
    labels = map(int, fid.read().split('\n'))

In [24]:
# Применяем на тестовых данных
predictions = pipe.predict(data_test)

In [27]:

accuracy_score(labels, predictions)

0.94

достаточно хороший скор

In [28]:
pred_df = pd.DataFrame(predictions, columns=['y'])
pred_df.index.name = 'Id'
pred_df.replace(0, 'neg', inplace=True)
pred_df.replace(1, 'pos', inplace=True)
pred_df.to_csv('predictions.csv', sep=',')